# Getting Started

This example showcases question answering over documents.
We have chosen this as the example for getting started because it nicely combines a lot of different elements (Text splitters, embeddings, vectorstores) and then also shows how to use them in a chain.

Question answering over documents consists of three steps:

1. Create an index
2. Create a question answering chain
3. Ask questions!

Each of the steps has multiple sub steps and potential configurations. In this notebook we will primarily focus on (1). We will start by showing the one-liner for doing so, but then break down what is actually going on.

First, let's import some common classes we'll use no matter what.

In [1]:
from langchain.chains import VectorDBQA
from langchain.llms import OpenAI

Next in the generic setup, let's specify the document loader we want to use.

In [2]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../state_of_the_union.txt')

## One Line Index Creation

To get started as quickly as possible, we can use the `VectorstoreIndexCreator`.

In [3]:
from langchain.indexes import VectorstoreIndexCreator

In [4]:
index = VectorstoreIndexCreator().from_loaders([loader])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


Now that the index is created, we can use it in a VectorDBQAChain to ask questions of the data!

In [5]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=index)
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a consensus builder, and has gained a broad range of support. He also said that she is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers."

## Walkthrough

Okay, so what's actually going on? How is this index getting created?

A lot of the magic is being hid in this `VectorstoreIndexCreator`. What is this doing?

There are three main steps going on after the documents are loaded:

1. Splitting documents into chunks
2. Creating embeddings for each document
3. Storing documents and embeddings in a vectorstore

Let's walk through this in code

In [6]:
documents = loader.load()

Next, we will split the documents into chunks.

In [8]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

We will then select which embeddings we want to use.

In [10]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

We now create the vectorstore to use as the index.

In [11]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(texts, embeddings)

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


Then, as before, we create a chain and use it to answer questions!

In [12]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=db)

In [13]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

" The President said that Ketanji Brown Jackson is one of the nation's top legal minds and a consensus builder, with a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. She is a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers."

`VectorstoreIndexCreator` is just a wrapper around all this logic. It is configurable in the text splitter it uses, the embeddings it uses, and the vectorstore it uses. For example, you can configure it as below:

In [14]:
index_creator = VectorstoreIndexCreator(
    vectorstore_cls=Chroma, 
    embedding=OpenAIEmbeddings(),
    text_splitter=CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
)

Hopefully this highlights what is going on under the hood of `VectorstoreIndexCreator`. While we think it's important to have a simple way to create indexes, we also think it's important to understand what's going on under the hood.